<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# 3.4.5 Evaluation

## Lesson Objectives

After completing this lesson you should be able to:

* Evaluate binary classification algorithms using area under the Receiver Operating Characteristic (ROC) curve
* Evaluate multiclass classification and regression algorithms using several metrics
* Evaluate logistic and linear regression algorithms using summaries

##Evaluators

After training a model and making predictions for the test data it is time to evaluate the model.
* An evaluator is a class that computes metrics from the predictions
* There are three types of evaluators available:
  * `BinaryClassificationEvaluator`
  * `MultiClassClassificationEvaluator`
  * `RegressionEvaluator` 

## Continuing from previous example

If you haven't downloaded the data set from the previous lesson then there is a link in the script to download it to your temporary folder and load it.

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0` // Or use any other 2.x version here
import  org.apache.spark.SparkContext
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
val sc= new SparkContext("local[*]","Linear Methods")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                     // Or use any other 2.x version here

import  org.apache.spark.SparkContext

import org.apache.log4j.{Level, Logger}

sc: SparkContext = org.apache.spark.SparkContext@41f3c78a

In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession

spark: SparkSession = org.apache.spark.sql.SparkSession@19b44ad
import spark.implicits._

import org.apache.spark.sql.functions._

In [3]:
import org.apache.spark.mllib.util.MLUtils.{
  convertVectorColumnsFromML => fromML,
  convertVectorColumnsToML => toML
}

import org.apache.spark.mllib.util.MLUtils.{
  convertVectorColumnsFromML => fromML,
  convertVectorColumnsToML => toML
}

In [4]:
import org.apache.spark.mllib.util.MLUtils
 
val data = toML(MLUtils.loadLibSVMFile(sc, "data/sample_libsvm_data.txt").toDF())

val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

import org.apache.spark.mllib.util.MLUtils
 

data: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]

## Example of Logistic Regression

Now we look at an example of binary classification using Logistic Regression. First I create a new instance of a Logistic Regression and set its parameters:

* The maximum number of iterations
* Regularization
* Elastic Net

In [5]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.classification.BinaryLogisticRegressionSummary

val logr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

val logrModel = logr.fit(trainingData)

println(s"Weights: ${logrModel.coefficients} Intercept: ${logrModel.intercept}")

19/12/19 13:52:31 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
19/12/19 13:52:31 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
19/12/19 13:52:32 INFO OWLQN: Step Size: 0.01023
19/12/19 13:52:32 INFO OWLQN: Val and Grad Norm: 0.665304 (rel: 0.0275) 1.15229
19/12/19 13:52:32 INFO OWLQN: Step Size: 1.000
19/12/19 13:52:32 INFO OWLQN: Val and Grad Norm: 0.620558 (rel: 0.0673) 0.773568
19/12/19 13:52:32 INFO OWLQN: Step Size: 1.000
19/12/19 13:52:32 INFO OWLQN: Val and Grad Norm: 0.610880 (rel: 0.0156) 0.382182
19/12/19 13:52:32 INFO OWLQN: Step Size: 1.000
19/12/19 13:52:32 INFO OWLQN: Val and Grad Norm: 0.604111 (rel: 0.0111) 0.254585
19/12/19 13:52:32 INFO OWLQN: Step Size: 1.000
19/12/19 13:52:32 INFO OWLQN: Val and Grad Norm: 0.601760 (rel: 0.00389) 0.227595
19/12/19 13:52:32 INFO OWLQN: Step Size: 0.5000
19/12/19 13:52:32 INFO OWLQN: Val and Grad Norm: 0.596657 (rel: 0.00848) 0.241448
19/12/19 13:52:33

Weights: (692,[244,272,300,328,345,346,373,378,401,406,407,427,428,433,434,440,455,456,461,462,468,483,484,489,490,496,511,512,523,539,540,550,551,568],[-9.882735218577889E-5,-1.6459188809518898E-4,-4.566644143167675E-4,-9.84000979370495E-6,-6.694567819262E-5,-4.350900866960413E-5,-5.526361932005142E-5,6.219246643958577E-5,-1.4847873754633821E-5,8.784146559109236E-4,1.1863864066241495E-4,-4.2700585947295114E-5,-1.1209504363120595E-4,3.8932769160215843E-4,0.0010467394925010825,-6.216301627160714E-5,-1.4426377508602758E-4,-1.9542835424619812E-4,6.002196488334172E-5,9.717846959151311E-4,-1.7790659664003415E-4,-1.6724410885496867E-4,-1.855320485158601E-4,5.683581231190236E-5,3.8989219698344396E-4,-4.901394739933316E-4,-2.0326756768035984E-4,-5.234266546945059E-4,-1.1905392061690462E-4,-1.0147509960301204E-4,-5.131815377979167E-4,-1.1616696535847838E-4,-6.628566733740409E-5,-1.7662239931089149E-4]) Intercept: 0.24548156736670593


import org.apache.spark.ml.classification.LogisticRegression

import org.apache.spark.ml.classification.BinaryLogisticRegressionSummary


logr: LogisticRegression = logreg_1ac6386ba362
logrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_1ac6386ba362, numClasses = 2, numFeatures = 692

summary

In [6]:
logrModel.summary.objectiveHistory

res5: Array[Double] = Array(
  0.6840977964176761,
  0.6653043394447086,
  0.6205579215485129,
  0.6108799139980405,
  0.6041107401429365,
  0.6017601436002061,
  0.5966565576843436,
  0.5943530883713057,
  0.5899299585158237,
  0.586756840396425,
  0.5857905141910918
)

## BinaryClassificationEvaluator

Let's start with the `BinaryClassificationEvaluator`:

* Evaluator for binary classification
* Expects two input columns: **rawPrediction** and **label**
* Supported metric: `areaUnderROC`

As its name states, it is used to evaluate binary classifiers. It expects two input columns, the `rawPrediction` column and the label column. The only supported metric is the area under the ROC curve.

This is an example of a Binary Classification Evaluator. I'm going to build upon the Logistic Regression model from the previous lesson and evaluate its predictions. First, I call the `transform` method on the test data to get a `DataFrame` with the predictions, which I name `predictionsLogR`:

In [7]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val predictionsLogR = logrModel.transform(testData)

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

predictionsLogR: org.apache.spark.sql.package.DataFrame = [label: double, features: vector ... 3 more fields]

Then, I create a new instance of a `BinaryClassificationEvaluator` and set the corresponding columns as inputs and the metric name to the only available metric, `areaUnderROC`:

In [8]:
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("rawPrediction").setMetricName("areaUnderROC")

evaluator: BinaryClassificationEvaluator = binEval_486c5f9c6188

Now I can call the evaluator's evaluate method on the predictions made by the Logistic Regression to get its area under the ROC curve:

In [9]:
val roc = evaluator.evaluate(predictionsLogR)

roc: Double = 1.0

## MulticlassClassificationEvaluator

Often, there are more than two categories you can classify an item into. The Multi-Class Classification Evaluator is an evaluator for multi-class classification problems.

* Expects two input columns: **prediction** and **label**
* Supported metrics:
  * `F1` (default)
  * Precision
  * Recall
  * `weightedPrecision`
  * `weightedRecall`

## Reusing RF Classification Example I

To show what a `Multiclass` Classification Evaluator can do we will need a model that can do more than the two categories the Random Forest classifier we calculated before would do. We will need to prepare the Pipeline for that model.
This is the exact script we have run in previous sessions to set up Pipelines for Random Forests and Gradient-Boosting Trees:

In [10]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer}

val labelIndexer = new StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(data)
val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)
val featureIndexer = new VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(data)

import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.feature.{StringIndexer, IndexToString, VectorIndexer}


labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_552853c717c0
labelConverter: IndexToString = idxToStr_f0b5c26d7959
featureIndexer: org.apache.spark.ml.feature.VectorIndexerModel = vecIdx_a06b1460e39c

In [11]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rfC = new RandomForestClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures").setNumTrees(3)

import org.apache.spark.ml.classification.RandomForestClassifier

import org.apache.spark.ml.classification.RandomForestClassificationModel


rfC: RandomForestClassifier = rfc_4a7d7fe7fefe

## Reusing RF Classification Example II

In [12]:
import org.apache.spark.ml.Pipeline

// split into training and test data
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

val pipelineRFC = new Pipeline().setStages(Array(labelIndexer, featureIndexer, rfC, labelConverter))
val modelRFC = pipelineRFC.fit(trainingData)
val predictionsRFC = modelRFC.transform(testData)

import org.apache.spark.ml.Pipeline

// split into training and test data

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
pipelineRFC: Pipeline = pipeline_f2d43ced89d7
modelRFC: org.apache.spark.ml.PipelineModel = pipeline_f2d43ced89d7
predictionsRFC: org.apache.spark.sql.package.DataFrame = [label: double, features: vector ... 6 more fields]

All the rest is exactly the same as before, calling the `fit` method to get a model
and calling the `transform` method to make predictions. The predictions are then
returned in the `predictionsRFC` `DataFrame`.

## MulticlassClassificationEvaluator

Now an example of a Multi Class Evaluator. For this example, I can evaluate any of the multiclass classifiers I have trained so far, and I choose to evaluate the predictions made by the Random Forest Classifier, which I previously assigned to the `predictionsRFC` `DataFrame`.

The true labels of the test set were in the indexed label column and the predictions made by the model were in its prediction column. So, I create a new instance of a `MulticlassClassificationEvaluator` and set the corresponding columns as inputs. Also, I set the metric to be **precision** instead of the default **F1-score**.

In [13]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val evaluator = new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("accuracy") 
val accuracy = evaluator.evaluate(predictionsRFC)

println("Test Error = " + (1.0 - accuracy))

Test Error = 0.0


import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


evaluator: MulticlassClassificationEvaluator = mcEval_8050c9fcaf32
accuracy: Double = 1.0

Now I can call the evaluator's evaluate method on the predictions made by the Random Forest Classifier to get the estimated precision, which is 96.6% or, put in another way, a 3.3% test error.

## RegressionEvaluator

* Evaluator for regression problems
* Expects two input columns: **prediction** and **label**
* Supported metrics:
  * **rmse**: root mean squared error (default)
  * **mse**: mean squared error
  * **r2**: R2, the coefficient of determination
  * **mae**: mean absolute error

## Reusing RF Regression Example

We will use the previous regression in our previous lesson in Random Forest. If you've come to this lesson directly and don't have the context, here is the code that produces the predictions we will evaluate:

In [14]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.regression.RandomForestRegressionModel

val rfR = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")
val pipelineRFR = new Pipeline().setStages(Array(featureIndexer, rfR))
val modelRFR = pipelineRFR.fit(trainingData)
val predictions = modelRFR.transform(testData)

import org.apache.spark.ml.regression.RandomForestRegressor

import org.apache.spark.ml.regression.RandomForestRegressionModel


rfR: RandomForestRegressor = rfr_1920995d6392
pipelineRFR: Pipeline = pipeline_ae46681c8425
modelRFR: org.apache.spark.ml.PipelineModel = pipeline_ae46681c8425
predictions: org.apache.spark.sql.package.DataFrame = [label: double, features: vector ... 2 more fields]

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.

In [15]:
sc.stop()